In [ ]:
import numpy as np
import glob 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os


from matplotlib.cm import ScalarMappable
from matplotlib.colors import ListedColormap, Normalize
# Load tables and and unionise 
from matplotlib.cm import ScalarMappable
from matplotlib.colors import  BoundaryNorm

In [ ]:
def create_polar_plot_from_df(df,cticks,ax=None, group_col='concentration',title=None):
    """
    Create a polar plot from a DataFrame containing co-occurrence probabilities.
    Assumes the DataFrame has: group_col, code, label, probability
    """
    #label_list=['¬C¬R¬F', '¬C¬RF', '¬CR¬F', '¬CRF', 'C¬R¬F', 'C¬RF', 'CR¬F', 'CRF']
    label_list=['','','','','','','','']

    # Ensure data is sorted by code for consistent angle mapping
    df_sorted = df.sort_values(by='code')

    # Unique positions and angle mapping
    unique_codes = sorted(df_sorted['code'].unique())
    n_codes = len(unique_codes)
    angles = np.linspace(0, 2 * np.pi, n_codes, endpoint=False)
    angles_closed = np.concatenate([angles, [angles[0]]])

    # Build label list (once, assume all groups have same codes/labels)
    #label_map = dict(zip(df_sorted['code'], label_list))
    #labels = [label_map[code] for code in unique_codes]

    # Assign colors for each group
    groups = df[group_col].unique()
    colors = plt.cm.jet(np.linspace(0, 1, len(groups))) # equally spaced colors

    #actualy plotting
    for i, group in enumerate(groups):
        group_df = df_sorted[df_sorted[group_col] == group]
        probs = group_df.set_index('code').loc[unique_codes]['mean'].values
        probs_closed = np.append(probs, probs[0])  # close the loop


        ax.plot(angles_closed, probs_closed, 'o-', linewidth=2, markersize=6,
                color=colors[i], label=str(group), alpha=0.85)
        m=ax.fill(angles_closed, probs_closed, alpha=0.1, color=colors[i])

    # Configure the polar axis
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)
    ax.set_thetagrids(angles * 180 / np.pi, label_list)
    
    # Set radius (y-axis) limits
    all_probs = df['mean'].values
    ax.set_ylim(0,1)
    ax.set_yticklabels([])
    if(title is not None):
        ax.set_title(title, fontsize=14, fontweight='bold', pad=20)
    ax.grid(True, alpha=0.6)
    ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.0))

    legend = ax.get_legend()
    if legend:
        legend.remove()



In [ ]:
def load_data(Folder,AB):
    coocc=pd.read_csv(f'{Folder}{AB}_cooccurance.csv',index_col=0)
    prob=pd.read_csv(f'{Folder}{AB}.csv',index_col=0)

    coocc['prob_neg_label_positive']=0
    coocc['total_positive']=0
    prob['total_positive']=prob['total_label_positive'].values * prob['prob_pos_label_positive'].values

    for d in prob.dataset.unique():
            for c in prob.chip.unique():
                coocc.loc[(coocc.dataset==d) & (coocc.chip==c),'prob_neg_label_positive']=prob.loc[(prob.dataset==d) & (prob.chip==c),'prob_neg_label_positive'].values[0]
                coocc.loc[(coocc.dataset==d) & (coocc.chip==c),'total_positive']=prob.loc[(prob.dataset==d) & (prob.chip==c),'total_positive'].values[0]

    return coocc

def calc_stat(data):
    av=data.groupby(by=['code','concentration']).probability.apply(lambda x: np.nanmean(x)).reset_index()
    std=data.groupby(by=['code','concentration']).probability.apply(lambda x: np.nanstd(x)).reset_index()
    std.rename(columns={'probability':'probability_std'},inplace=True)
    av=av.merge(std,on=['code','concentration'])

    av_prob=data.groupby(by=['code','concentration']).prob_neg_label_positive.apply(lambda x: np.nanmean(x)).reset_index()
    av=av.merge(av_prob,on=['code','concentration'])

    return av


def calc_smooted_profiles(data,prop,prop_for_window,window_size=1,suppress_smaller=None):

    '''
        data is dataframe, prop is the property that we want to average, and we average it on a sliding window of prop_for_window
        where the actual windowsize is 2 * window_size (symmetric)
        output is a dictionary with the average and SEM
    '''
    
    if(suppress_smaller is not None):
        data_to_average=data.drop(data[data['total_positive']<suppress_smaller].index)
    else:
        data_to_average=data


    data_to_average=data_to_average.loc[data_to_average.code==prop,['probability',prop_for_window]].to_numpy()
    val_mean=[]
    val_std=[]


    grid_for_window=np.unique(data_to_average[:,1]) # 
                    
    for c in grid_for_window:
        values_in_window=data_to_average[(data_to_average[:,1]>=c-window_size) & (data_to_average[:,1]<=c+window_size) ,0]
        if(len(values_in_window)==0):
            val_mean.append(np.nan)
            val_std.append(np.nan)
        else:   
            val_mean.append(np.nanmean(values_in_window))
            val_std.append(np.nanstd(values_in_window)/np.sqrt(len(values_in_window)))


    dic={'code':prop ,'mean': np.array(val_mean),'std': np.array(val_std),prop_for_window:grid_for_window}  
  

    return dic



In [ ]:
Folder='tables/probability_tables/'



Genta=load_data(Folder,'Gentamicin')
Cipro=load_data(Folder,'Ciprofloxacin')
Amp=load_data(Folder,'Ampicilin')

Tetra=load_data(Folder,'Tetracycline')
Chp=load_data(Folder,'Chloramphenicol')

df=[Genta,Cipro,Amp,Tetra,Chp]

In [ ]:
window_size=[0.15,3.9,1.8,21,1.1]  #weighted mean for concentration (actually half but internally the window will be symmetric)

In [ ]:
collect_dict=[]
for c in Genta.code.unique():
    collect_dict.append(calc_smooted_profiles(Genta,c,'concentration',window_size[0],suppress_smaller=3))

window_Genta=pd.DataFrame(collect_dict)
window_Genta = window_Genta.explode(['mean', 'std', 'concentration'])   


collect_dict=[]
for c in Cipro.code.unique():
    collect_dict.append(calc_smooted_profiles(Cipro,c,'concentration',window_size[1],suppress_smaller=3))

window_Cipro=pd.DataFrame(collect_dict)
window_Cipro = window_Cipro.explode(['mean', 'std', 'concentration'])    


collect_dict=[]
for c in Amp.code.unique():
    collect_dict.append(calc_smooted_profiles(Amp,c,'concentration',window_size[2],suppress_smaller=3))

window_Amp=pd.DataFrame(collect_dict)
window_Amp = window_Amp.explode(['mean', 'std', 'concentration'])    


collect_dict=[]
for c in Tetra.code.unique():
    collect_dict.append(calc_smooted_profiles(Tetra,c,'concentration',window_size[3],suppress_smaller=3))

window_Tetra=pd.DataFrame(collect_dict)
window_Tetra = window_Tetra.explode(['mean', 'std', 'concentration'])    

collect_dict=[]
for c in Chp.code.unique():
    collect_dict.append(calc_smooted_profiles(Chp,c,'concentration',window_size[4],suppress_smaller=3))

window_Chp=pd.DataFrame(collect_dict)
window_Chp = window_Chp.explode(['mean', 'std', 'concentration'])


df_window=[window_Genta,window_Cipro,window_Amp,window_Tetra,window_Chp]

# Plot

In [ ]:
xlim=[[-0.07,2.04],[-1.5,34],[-1,21],[-5,365],[-1,21]]
xticks=[np.arange(0,10,0.25),np.arange(0,50,4),np.arange(0,50,2.5),np.arange(0,400,45),np.arange(0,100,2.5)]


colors=sns.color_palette()
markers=['o','s','<','v','h','P','*','^','8','p','H','d','D','X']
msize=80
lw_big=4
lw_small=2


clim=[[-0.07,2.04],[-1.5,34],[-1,21],[-5,365],[-1,21]]
cticks=[np.arange(0,10,0.25),np.arange(0,50,4),np.arange(0,50,2.5),np.arange(0,400,45),np.arange(0,100,2.5)]

In [ ]:
fig, axs = plt.subplots(1,5, figsize=[20, 10],dpi=200,subplot_kw=dict(projection='polar') )
for i in range(5): # for every antibiotic
    create_polar_plot_from_df(df_window[i],cticks[i],axs[i],'concentration')
plt.rcParams.update()

fig.subplots_adjust(left=0, right=1, top=1, bottom=0, wspace=0.2)

# Colorbar

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.colorbar import ColorbarBase

# Create figure
fig, ax = plt.subplots(figsize=(8, 1))

# Setup colormap and range
cmap = plt.cm.jet
norm = mcolors.Normalize(vmin=0, vmax=100)

# Create colorbar
cb = ColorbarBase(ax, cmap=cmap, norm=norm, orientation='horizontal')
cb.set_label('', fontsize=12)
cb.set_ticklabels([])
cb.set_ticks([])

# Export
plt.tight_layout()
plt.savefig('my_colorbar.eps', dpi=300, bbox_inches='tight', 
            facecolor='white', edgecolor='none')
plt.show()